# Circuit learning module: Pennylane with SPSA

This code is based on [Quanthoven](https://github.com/CQCL/Quanthoven/blob/main/experiment.ipynb) and [Pennylane demo](https://pennylane.ai/qml/demos/spsa.html).

In [1]:
import json
import os
import glob
import warnings
from pathlib import Path
import numpy as np
import pickle
import pennylane as qml
from sympy import default_sort_key
import torch
from discopy.quantum.pennylane import to_pennylane, PennyLaneCircuit
from inspect import signature
from noisyopt import minimizeSPSA

this_folder = os.path.abspath(os.getcwd())

## Read circuit data

We read the circuits from the pickled files.

In [2]:
training_circuits_paths = glob.glob(this_folder + "//simplified-JOB-diagrams//circuits//binary_classification//training//[0-9]*.p")
test_circuits_paths = glob.glob(this_folder + "//simplified-JOB-diagrams//circuits//binary_classification//test//[0-9]*.p")

def read_diagrams(circuit_paths):
    circuits = {}
    for serialized_diagram in circuit_paths:
        base_name = Path(serialized_diagram).stem
        f = open(serialized_diagram, "rb")
        diagram = pickle.load(f)
        circuits[base_name] = diagram
    return circuits


training_circuits = read_diagrams(training_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)


print("Number of training circuits: ", len(training_circuits))
print("Number of test circuits: ", len(test_circuits))

Number of training circuits:  25
Number of test circuits:  10


## Read training and test data

In [3]:
training_data, test_data = None, None
with open(this_folder + "//data//training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(this_folder + "//data//test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
    

def time_to_states(data):
    labeled_data = {}
    for elem in data:
        if elem["time"] < 2001:
            labeled_data[elem["name"]] = [1, 0] # corresponds to |0>
        else:
            labeled_data[elem["name"]] = [0, 1] # corresponds to |1>
    return labeled_data


training_data_labels = time_to_states(training_data)
test_data_labels = time_to_states(test_data)

training_circuits_l, training_data_labels_l = [], []
test_circuits_l, test_data_labels_l = [], []

for key in training_circuits:
    training_circuits_l.append(training_circuits[key])
    training_data_labels_l.append(training_data_labels[key])

for key in test_circuits:
    test_circuits_l.append(test_circuits[key])
    test_data_labels_l.append(test_data_labels[key])

## Construct circuits in Pennylane

Next we translate DisCoPy circuits in Pennylane and rewrite the Pennylane circuits. By default the encoded measurement function in the Pennylane circuit function is `state()` or `probs()` which we change to `sampe()`. We use `sample()` because we need to do post-selection which is not implemented in Pennylane. Technically we also have the problem that Pennylane does not natively support training over circuits which have varying signature (parameters) and varying number of wires. Thus we rewrite the circuits so that at each step, all the circuits accept all the parameters as input but they will utilize only a subset of them. Because the parameters need to be inputed as a vector, for each circuit we calculate a signature tuple, which encodes the information of the needed parameters in the circuit.

The code assumes that the DisCoPy circuit contains only such gates that accept a single parameter or no parameters.

$$ \texttt{parameter symbol in circuit} \to \texttt{index among all parameters in model} \to \texttt{value from input for circuit} $$

In [4]:
tot_qubits = 20
dev = qml.device("default.qubit", wires=tot_qubits, shots=1000)

qml_circuits_training = []
qml_circuits_test = []
symbols = set([elem for c in training_circuits for elem in training_circuits[c].free_symbols])
symbols = list(sorted(symbols, key=default_sort_key))

for circuit_diagram in training_circuits_l:
    pennylane_circuit = to_pennylane(circuit_diagram)
    params = pennylane_circuit.params
    pennylane_wires = pennylane_circuit.wires
    ops = pennylane_circuit.ops
    param_symbols = [[sym[0].as_ordered_factors()[2]] if len(sym) > 0 else [] for sym in params]
    symbol_to_index = {}

    for sym in param_symbols:
        if len(sym) > 0:
            symbol_to_index[sym[0]] = symbols.index(sym[0])

    @qml.qnode(dev)
    def qml_circuit(circ_params):
        for op, param, wires in zip(ops, param_symbols, pennylane_wires):
            if len(param) > 0:
                param = param[0]
                op(circ_params[symbol_to_index[param]], wires = wires)
            else:
                op(wires = wires)
        return qml.sample()
    
    qml_circuits_training.append(qml_circuit)

training_qnodes = qml.QNodeCollection(qml_circuits_training)

## Post-selection

In [5]:
def post_select_shot(shot, post_selection):
    diff = len(shot) - len(post_selection)
    for i in range(len(post_selection)):
        if shot[i + diff] != post_selection[i]:
            return []
    else:
        return shot
    
def post_select_circuit_samples(circuit_samples):
    selected_samples = []
    for circuit_sample in circuit_samples:
        selected = []
        for shot in circuit_sample:
            post_selected = post_select_shot(shot, [0]*(len(shot) - 1))
            if len(post_selected) > 0:
                if post_selected[0] == 0:
                    selected.append([1, 0])
                else:
                    selected.append([0, 1])
        selected_samples.append(selected)
    return selected_samples

## Cost function

In [6]:
#rng = np.random.default_rng(0)
#init_params_spsa = np.array(rng.random(len(symbols)))
#cost_spsa(init_params_spsa)

def most_common(lst):
    counts = {}
    for e in lst:
        if tuple(e) in counts:
            counts[tuple(e)] += 1
        else:
            counts[tuple(e)] = 1
    return [counts[(1,0)]/len(lst), counts[(0,1)]/len(lst)] 

#rng = np.random.default_rng(SEED)
#init_params_spsa = np.array(rng.random(len(symbols)))
#circuit_samples = training_qnodes(init_params_spsa)
#predictions = post_select_circuit_samples(circuit_samples)
#print(predictions[0])
#print(most_common(predictions[0]))

In [7]:
def make_cost_fn(qnodes, labels):    
    def cost_spsa(params, **kwargs):
        #circuit_samples = training_qnodes(params)
        measurement = qnodes(params)
        circuit_samples = post_select_circuit_samples(measurement)
        predictions = []
        for circuit_sample in circuit_samples:
            expected = most_common(circuit_sample)
            predictions.append(expected)
        
        #print(predictions)
        cost = -np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        acc = np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(acc)
        
        return cost
    
    costs, accuracies = [], []
    return cost_spsa, costs, accuracies

## SPSA minimization

In [8]:
BATCH_SIZE = 32
EPOCHS = 200
SEED = 0

rng = np.random.default_rng(SEED)
init_params_spsa = np.array(rng.random(len(symbols)))
numpy.random.seed(SEED)

train_cost_fn, train_costs, train_accs = make_cost_fn(training_qnodes, training_data_labels_l)
#dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, dev_labels)

# Evaluate the initial cost
cost_store_spsa = [train_cost_fn(init_params_spsa)]
#print(init_params_spsa)
#print()
#print(training_data_labels_l)
#print()
#print(cost_store_spsa)
device_execs_spsa = [0]

def callback_fn(xk):
    cost_val = train_cost_fn(xk)
    cost_store_spsa.append(cost_val)

    # We've evaluated the cost function, let's make up for that
    num_executions = int(dev.num_executions / 2)
    device_execs_spsa.append(num_executions)

    iteration_num = len(cost_store_spsa)
    print(
            f"Params = {xk}, "
            f"Iteration = {iteration_num}, "
            f"Number of device executions = {num_executions}, "
            f"Cost = {cost_val}"
        )

In [ ]:
result = minimizeSPSA(train_cost_fn, x0=init_params_spsa.copy(), a=0.91, c=0.12, niter=EPOCHS, paired=False, callback=callback_fn)

#res = minimizeSPSA(
#    cost_spsa,
#    x0=init_params_spsa.copy(),
#    niter=EPOCHS,
#    paired=False,
#    c=0.15,
#    a=0.2,
#    callback=callback_fn,
#)

Params = [0.60104737 0.2338724  0.00505921 ... 0.89903452 0.84539286 0.47479219], Iteration = 2, Number of device executions = 50, Cost = 0.7120762862749391
Params = [ 0.69515929  0.13976048 -0.08905271 ...  0.8049226   0.93950478
  0.38068027], Iteration = 3, Number of device executions = 87, Cost = 0.6615862146685579
Params = [ 0.78384029  0.05107949 -0.1777337  ...  0.71624161  1.02818577
  0.46936127], Iteration = 4, Number of device executions = 125, Cost = 0.7460381406386465
Params = [ 0.78922026  0.04569951 -0.17235373 ...  0.72162158  1.03356575
  0.46398129], Iteration = 5, Number of device executions = 162, Cost = 0.7107609971686729
Params = [ 0.78303381  0.05188596 -0.16616728 ...  0.72780803  1.0273793
  0.47016774], Iteration = 6, Number of device executions = 200, Cost = 0.6668101021206293
Params = [ 0.76154107  0.03039322 -0.14467454 ...  0.74930077  1.00588656
  0.448675  ], Iteration = 7, Number of device executions = 237, Cost = 0.707839039079418
Params = [ 0.73653022

In [ ]:
import matplotlib.pyplot as plt

fig, ((ax_tl, ax_tr), (ax_bl, ax_br)) = plt.subplots(2, 2, sharex=True, sharey='row', figsize=(10, 6))
ax_tl.set_title('Training set')
ax_tr.set_title('Development set')
ax_bl.set_xlabel('Iterations')
ax_br.set_xlabel('Iterations')
ax_bl.set_ylabel('Accuracy')
ax_tl.set_ylabel('Loss')

colours = iter(plt.rcParams['axes.prop_cycle'].by_key()['color'])
ax_tl.plot(train_costs[1::2], color=next(colours))  # training evaluates twice per iteration
ax_bl.plot(train_accs[1::2], color=next(colours))   # so take every other entry
ax_tr.plot(dev_costs, color=next(colours))
ax_br.plot(dev_accs, color=next(colours))

# print test accuracy
test_cost_fn, _, test_accs = make_cost_fn(test_pred_fn, test_labels)
test_cost_fn(result.x)
print('Test accuracy:', test_accs[0])